In [1]:
"""applay regularizers"""
import numpy as np
from tensorflow_core.python.keras.api._v2.keras.datasets import mnist
from tensorflow_core.python.keras.api._v2.keras.models import Sequential
from tensorflow_core.python.keras.api._v2.keras.layers import Dense, Dropout, Activation
from tensorflow_core.python.keras.api._v2.keras.optimizers import SGD, RMSprop, Adam
from tensorflow_core.python.keras.api._v2.keras.utils import to_categorical
import tensorflow_core.python.keras.api._v2.keras.regularizers as regularizers
from make_tensorboard import make_tensorboard

In [2]:
# 乱数の固定
np.random.seed(1671)

In [3]:
# パラメータ設定
NB_EPOCH = 20           # エポック数
BATCH_SIZE = 128        # バッチサイズ
VERBOSE = 1             #
NB_CLASSES = 10         # 出力数
# OPTIMIZER = RMSprop()      # 最適化関数
OPTIMIZER = Adam()      # 最適化関数
N_HIDDEN = 128          # 隠れ層数
VALIDATION_SPLIT = 0.2  # 検証データの割合
DROPOUT = 0.3  # ← 追加

In [4]:
# データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()
RESHAPED = 784  # 28×28 = 784

In [5]:
# データの整形
TRAIN_DATA_SIZE = 60000
TEST_DATA_SIZE = 10000
X_train = X_train.reshape(TRAIN_DATA_SIZE, RESHAPED)
X_test = X_test.reshape(TEST_DATA_SIZE, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [6]:
# 正規化
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train sample')
print(X_test.shape[0], 'test sample')

60000 train sample
10000 test sample


In [7]:
# ラベルをOne-hotエンコーディング
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

In [14]:
# Sequentialモデルを定義
model = Sequential()
# 層を定義
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))  # ← 784 × 128
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))  # ← 追加
model.add(Dense(N_HIDDEN, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))                           # ← 128 × 128
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))  # ← 追加
model.add(Dense(NB_CLASSES, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))                         # ← 128 × 10
model.add(Activation('softmax'))
# モデルを評価しTensorBoard互換のログに書き込む
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [15]:
# モデルをコンパイル
# -損失関数　：カテゴリカルクロスエントロピー
# -最適化関数：確率的勾配降下法
# -評価関数　：Accuracy(正解率)
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [16]:
# tensorboard用
callbacks = [make_tensorboard(set_dir_name='keras_MINST_V5')]

In [17]:
# 学習
# -バッチサイズ　　：128
# -エポック数　　　：200
# -コールバック関数：make_tensorboard(訓練の各段階で呼び出される関数)
# -ＶＥＲＢＯＳＥ　：詳細表示モードON
# -検証データの割合：20%
model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE,
          epochs=NB_EPOCH,
          callbacks=callbacks,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 3s 54us/sample - loss: 1.6259 - accuracy: 0.8295 - val_loss: 1.0464 - val_accuracy: 0.9129
Epoch 2/20
48000/48000 [==============================] - 2s 33us/sample - loss: 1.1448 - accuracy: 0.8663 - val_loss: 0.9577 - val_accuracy: 0.9202
Epoch 3/20
48000/48000 [==============================] - 2s 34us/sample - loss: 1.0873 - accuracy: 0.8734 - val_loss: 0.9092 - val_accuracy: 0.9249
Epoch 4/20
48000/48000 [==============================] - 2s 33us/sample - loss: 1.0535 - accuracy: 0.8771 - val_loss: 0.8872 - val_accuracy: 0.9293
Epoch 5/20
48000/48000 [==============================] - 2s 33us/sample - loss: 1.0336 - accuracy: 0.8799 - val_loss: 0.8721 - val_accuracy: 0.9296
Epoch 6/20
48000/48000 [==============================] - 2s 33us/sample - loss: 1.0205 - accuracy: 0.8813 - val_loss: 0.8683 - val_accuracy: 0.9299
Epoch 7/20
48000/48000 [==============================] 

In [18]:
# スコアを表示
score = model.evaluate(X_test, Y_test, verbose=0)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])


Test score: 0.7959384333610535
Test accuracy: 0.938


In [13]:
'''RMSprop
2s 34us/sample - loss: 0.0696 - accuracy: 0.9798 - val_loss: 0.1060 - val_accuracy: 0.9778
Test score: 0.1014234864374066
Test accuracy: 0.9786
'''

'RMSprop\n2s 34us/sample - loss: 0.0696 - accuracy: 0.9798 - val_loss: 0.1060 - val_accuracy: 0.9778\nTest score: 0.1014234864374066\nTest accuracy: 0.9786\n'